In [ ]:
from google.colab import drive
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import csv

from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier

In [ ]:
drive.mount('/drive') 

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
train_Les = np.load('/drive/My Drive/CAD_1_1/train_Les_hairRemoved.npz',allow_pickle=True)
train_NV  = np.load('/drive/My Drive/CAD_1_1/train_NV_hairRemoved.npz',allow_pickle=True)
val_Les   = np.load('/drive/My Drive/CAD_1_1/val_Les_hairRemoved.npz',allow_pickle=True)
val_NV    = np.load('/drive/My Drive/CAD_1_1/val_NV_hairRemoved.npz',allow_pickle=True)
test    = np.load('/drive/My Drive/CAD_1_1/test_hairRemoved.npz',allow_pickle=True)

In [ ]:
trainX_Les  = np.array(train_Les['arr_0'])
trainX_NV   = np.array(train_NV['arr_0'])
valX_Les    = np.array(val_Les['arr_0'])
valX_NV     = np.array(val_NV['arr_0'])
test     = np.array(test['test'])


In [ ]:
print(trainX_Les.shape)
print(trainX_NV.shape)
print(valX_Les.shape)
print(valX_NV.shape)
print(test.shape)


(2400, 450, 600, 3)
(2400, 450, 600, 3)
(600, 450, 600, 3)
(600, 450, 600, 3)
(1015, 450, 600, 3)


GLCM(gray) Features

In [ ]:
import skimage
import skimage.feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
def GLCM(image):
  GLCM_feature_data = np.zeros((1,6))
  img=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  glcm = greycomatrix(img, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
  GLCM_feature_data[0][0]= greycoprops(glcm, 'contrast')
  GLCM_feature_data[0][1]= greycoprops(glcm, 'correlation')
  GLCM_feature_data[0][2]= greycoprops(glcm, 'dissimilarity')
  GLCM_feature_data[0][3]= greycoprops(glcm, 'energy')
  GLCM_feature_data[0][4]= greycoprops(glcm, 'ASM')
  GLCM_feature_data[0][5]= greycoprops(glcm, 'homogeneity')
  return GLCM_feature_data


In [ ]:
GLCM_Features_trainX_Les  = []
GLCM_Features_trainX_NV   = []
GLCM_Features_valX_Les    = []
GLCM_Features_valX_NV     = []

for x in tqdm(trainX_Les):
  des = GLCM(x)
  if des is not None:
    GLCM_Features_trainX_Les.append(des)

for x in tqdm(trainX_NV):
  des = GLCM(x)
  if des is not None:
    GLCM_Features_trainX_NV.append(des)

for x in tqdm(valX_Les):
  des = GLCM(x)
  if des is not None:
    GLCM_Features_valX_Les.append(des)

for x in tqdm(valX_NV):
  des = GLCM(x)
  if des is not None:
    GLCM_Features_valX_NV.append(des)

100%|██████████| 600/600 [00:05<00:00, 103.76it/s]


In [ ]:
GLCM_Features_trainX_Les=np.squeeze(np.array(GLCM_Features_trainX_Les))
GLCM_Features_trainX_NV=np.squeeze(np.array(GLCM_Features_trainX_NV))
GLCM_Features_valX_Les=np.squeeze(np.array(GLCM_Features_valX_Les))
GLCM_Features_valX_NV=np.squeeze(np.array(GLCM_Features_valX_NV))
print(GLCM_Features_trainX_Les.shape)
print(GLCM_Features_trainX_NV.shape)
print(GLCM_Features_valX_Les.shape)
print(GLCM_Features_valX_NV.shape)

(2400, 6)
(2400, 6)
(600, 6)
(600, 6)


In [ ]:
GLCM_train=np.vstack((GLCM_Features_trainX_Les,GLCM_Features_trainX_NV))
GLCM_val=np.vstack((GLCM_Features_valX_Les,GLCM_Features_valX_NV))

COLOR Feature 

In [ ]:
def color_moments(image):
  color_feature_data = np.zeros((1,12))
  color_feature_data[0][0] = np.mean(image[:,:,0])
  color_feature_data[0][1] = np.mean(image[:,:,1])
  color_feature_data[0][2]  = np.mean(image[:,:,2])
  color_feature_data[0][3]  = np.std(image[:,:,0])
  color_feature_data[0][4]   = np.std(image[:,:,1])
  color_feature_data[0][5]   = np.std(image[:,:,2])
  color_feature_data[0][6]  = skew(image[:,:,0].reshape(-1))
  color_feature_data[0][7]  = skew(image[:,:,1].reshape(-1))
  color_feature_data[0][8]  = skew(image[:,:,2].reshape(-1))
  color_feature_data[0][9]  = kurtosis(image[:,:,0].reshape(-1))
  color_feature_data[0][10]  = kurtosis(image[:,:,1].reshape(-1))
  color_feature_data[0][11] = kurtosis(image[:,:,2].reshape(-1))
  return color_feature_data

In [ ]:
import numpy as np
from scipy.stats import skew
from scipy.stats import kurtosis
color_test  = []
for x in tqdm(test):
  des = color_moments(x)
  if des is not None:
    color_test.append(des)

100%|██████████| 1015/1015 [00:40<00:00, 25.02it/s]


In [ ]:
np.array(color_test).shape

(1015, 1, 12)

In [ ]:
color_Test=np.squeeze(np.array(color_test))

In [ ]:
color_Test.shape

(1015, 12)

In [ ]:
import numpy as np
from scipy.stats import skew
from scipy.stats import kurtosis
color_Features_trainX_Les  = []
color_Features_trainX_NV   = []
color_Features_valX_Les    = []
color_Features_valX_NV     = []

for x in tqdm(trainX_Les):
  des = color_moments(x)
  if des is not None:
    color_Features_trainX_Les.append(des)

for x in tqdm(trainX_NV):
  des = color_moments(x)
  if des is not None:
    color_Features_trainX_NV.append(des)

for x in tqdm(valX_Les):
  des = color_moments(x)
  if des is not None:
    color_Features_valX_Les.append(des)

for x in tqdm(valX_NV):
  des = color_moments(x)
  if des is not None:
    color_Features_valX_NV.append(des)


100%|██████████| 600/600 [00:46<00:00, 13.04it/s]


In [ ]:
color_Features_trainX_Les=np.squeeze(np.array(color_Features_trainX_Les))
color_Features_trainX_NV=np.squeeze(np.array(color_Features_trainX_NV))
color_Features_valX_Les=np.squeeze(np.array(color_Features_valX_Les))
color_Features_valX_NV=np.squeeze(np.array(color_Features_valX_NV))
print(color_Features_trainX_Les.shape)
print(color_Features_trainX_NV.shape)
print(color_Features_valX_Les.shape)
print(color_Features_valX_NV.shape)

(2400, 12)
(2400, 12)
(600, 12)
(600, 12)


In [ ]:
color_train=np.vstack((color_Features_trainX_Les,color_Features_trainX_NV))
color_val=np.vstack((color_Features_valX_Les,color_Features_valX_NV))

GLCM (All Channels)

In [ ]:
def GLCM_features(image, bit_depth=8):
  GLCM_feature_data = np.zeros((1,12))
  GLCM_0  = greycomatrix(image[:,:,0],  [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=2**bit_depth)
  GLCM_1  = greycomatrix(image[:,:,1],  [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=2**bit_depth)
  GLCM_2  = greycomatrix(image[:,:,2],  [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=2**bit_depth)
  GLCM_feature_data[0][0]  = greycoprops(GLCM_0,  'contrast').mean()
  GLCM_feature_data[0][1]  = greycoprops(GLCM_1,  'contrast').mean()
  GLCM_feature_data[0][2]  = greycoprops(GLCM_2,  'contrast').mean()
  GLCM_feature_data[0][3]    = greycoprops(GLCM_0,  'dissimilarity').mean()
  GLCM_feature_data[0][4]    = greycoprops(GLCM_1,  'dissimilarity').mean()
  GLCM_feature_data[0][5]    = greycoprops(GLCM_2,  'dissimilarity').mean()
  GLCM_feature_data[0][6]    = greycoprops(GLCM_0,  'correlation').mean()
  GLCM_feature_data[0][7]    = greycoprops(GLCM_1,  'correlation').mean()
  GLCM_feature_data[0][8]    = greycoprops(GLCM_2,  'correlation').mean()
  GLCM_feature_data[0][9]      = greycoprops(GLCM_0,  'homogeneity').mean()
  GLCM_feature_data[0][10]      = greycoprops(GLCM_1,  'homogeneity').mean()
  GLCM_feature_data[0][11]      = greycoprops(GLCM_2,  'homogeneity').mean()
  return GLCM_feature_data

In [ ]:
import numpy as np
from scipy.stats import skew
from scipy.stats import kurtosis
import skimage.feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
glcm_Features_trainX_Les  = []
glcm_Features_trainX_NV   = []
glcm_Features_valX_Les    = []
glcm_Features_valX_NV     = []
glcmHSV_Features_trainX_Les  = []
glcmHSV_Features_trainX_NV   = []
glcmHSV_Features_valX_Les    = []
glcmHSV_Features_valX_NV     = []
glcmYCR_Features_trainX_Les  = []
glcmYCR_Features_trainX_NV   = []
glcmYCR_Features_valX_Les    = []
glcmYCR_Features_valX_NV     = []

for x in tqdm(trainX_Les):
  RGB    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  HSV    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
  YCR  = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB))
  DES  = np.concatenate((RGB,HSV,YCR),axis=0)
  glcm_Features_trainX_Les.append(DES)

for x in tqdm(trainX_NV):
  RGB    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  HSV    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
  YCR  = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB))
  DES  = np.concatenate((RGB,HSV,YCR),axis=0)
  glcm_Features_trainX_NV.append(DES)

for x in tqdm(valX_Les):
  RGB    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  HSV    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
  YCR  = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB))
  DES  = np.concatenate((RGB,HSV,YCR),axis=0)
  glcm_Features_valX_Les.append(DES)

for x in tqdm(valX_NV):
  RGB    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  HSV    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
  YCR  = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB))
  DES  = np.concatenate((RGB,HSV,YCR),axis=0)
  glcm_Features_valX_NV.append(DES)

100%|██████████| 600/600 [03:01<00:00,  3.30it/s]


In [ ]:
import numpy as np
from scipy.stats import skew
from scipy.stats import kurtosis
import skimage.feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops

glcm_test    = []
for x in tqdm(test):
  RGB    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  HSV    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
  YCR  = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB))
  DES  = np.concatenate((RGB,HSV,YCR),axis=0)
  glcm_test.append(DES)

glcm_test
for x in tqdm(test):
  RGB    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2RGB))
  glcm_test.append(RGB)
  HSV    = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2HSV))
  glcm_test.append(HSV)
  YCR  = GLCM_features(cv2.cvtColor(x, cv2.COLOR_BGR2YCR_CB))
  glcm_test.append(YCR)


100%|██████████| 1015/1015 [22:46<00:00,  1.35s/it]


In [ ]:
glcm_test_array=np.array(glcm_test)


In [ ]:
newarr1 = glcm_test_array.reshape(glcm_test_array.shape[0], (glcm_test_array.shape[1]*glcm_test_array.shape[2]))
print(newarr1.shape)

(1015, 36)


In [ ]:
newarr1 = glcm_Features_trainX_Les.reshape(glcm_Features_trainX_Les.shape[0], (glcm_Features_trainX_Les.shape[1]*glcm_Features_trainX_Les.shape[2]))
newarr2 = glcm_Features_trainX_NV.reshape(glcm_Features_trainX_NV.shape[0], (glcm_Features_trainX_NV.shape[1]*glcm_Features_trainX_NV.shape[2]))
newarr3 = glcm_Features_valX_Les.reshape(glcm_Features_valX_Les.shape[0], (glcm_Features_valX_Les.shape[1]*glcm_Features_valX_Les.shape[2]))
newarr4 = glcm_Features_valX_NV.reshape(glcm_Features_valX_NV.shape[0], (glcm_Features_valX_NV.shape[1]*glcm_Features_valX_NV.shape[2]))
print(np.array(newarr1).shape)
print(np.array(newarr2).shape)
print(np.array(newarr3).shape)
print(np.array(newarr4).shape)

(2400, 36)
(2400, 36)
(600, 36)
(600, 36)


In [ ]:
np.vstack(glcm_test)

In [ ]:
glcm_train=np.vstack((newarr1,newarr2))
glcm_val=np.vstack((newarr3,newarr4))
print(glcm_train.shape)

(4800, 36)


In [ ]:
glcm_Features_trainX_Les=np.squeeze(np.array(glcm_Features_trainX_Les))
glcm_Features_trainX_NV=np.squeeze(np.array(glcm_Features_trainX_NV))
glcm_Features_valX_Les=np.squeeze(np.array(glcm_Features_valX_Les))
glcm_Features_valX_NV=np.squeeze(np.array(glcm_Features_valX_NV))
print(glcm_Features_trainX_Les.shape)
print(glcm_Features_trainX_NV.shape)
print(glcm_Features_valX_Les.shape)
print(glcm_Features_valX_NV.shape)

(2400, 3, 12)
(2400, 3, 12)
(600, 3, 12)
(600, 3, 12)


In [ ]:
GLCM_Features_trainX_Les=np.squeeze(np.array(GLCM_Features_trainX_Les))
GLCM_Features_trainX_NV=np.squeeze(np.array(GLCM_Features_trainX_NV))
GLCM_Features_valX_Les=np.squeeze(np.array(GLCM_Features_valX_Les))
GLCM_Features_valX_NV=np.squeeze(np.array(GLCM_Features_valX_NV))
print(GLCM_Features_trainX_Les.shape)
print(GLCM_Features_trainX_NV.shape)
print(GLCM_Features_valX_Les.shape)
print(GLCM_Features_valX_NV.shape)

(2400, 6)
(2400, 6)
(600, 6)
(600, 6)


In [ ]:
GLCM_train=np.vstack((GLCM_Features_trainX_Les,GLCM_Features_trainX_NV))
GLCM_val=np.vstack((GLCM_Features_valX_Les,GLCM_Features_valX_NV))

In [ ]:
train=GLCM_train
val=GLCM_val

In [ ]:
import pandas as pd

trainData = pd.DataFrame(color_train)
trainData['labels']=0
trainData['labels'][0:2400]=1
valData = pd.DataFrame(color_val)
valData['labels']=0
valData['labels'][0:600]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
testData = pd.DataFrame(color_Test)


In [ ]:
testData.to_csv('/drive/My Drive/CAD_1_1/goodcolor_test.csv',index=False)

In [ ]:
trainDataShuffle=trainData.sample(frac=1)
valDataShuffle=valData.sample(frac=1)

In [ ]:
trainData.to_csv('/drive/My Drive/CAD_1_1/goodcolor_train.csv',index=False)
valData.to_csv('/drive/My Drive/CAD_1_1/goodcolor_val.csv',index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rndst=RandomForestClassifier(n_estimators=100)
rndst.fit(trainData.drop(["labels"],axis=1), trainData['labels'])
predict=rndst.predict(valData.drop(["labels"],axis=1))
#rndst.score(predict,valData['labels'])
print(metrics.accuracy_score(valData['labels'],predict))



In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
svc=SVC()
svc.fit(trainData.drop(["labels"],axis=1), trainData['labels'])
predict=svc.predict(valData.drop(["labels"],axis=1))
#rndst.score(predict,valData['labels'])
print(metrics.accuracy_score(valData['labels'],predict))

0.7808333333333334
